<a href="https://colab.research.google.com/github/EAkeweje/Accelerating-Hydrogen-Oxidation-Calculations/blob/main/Main_ModelOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb -Uq

In [2]:
#import and login
import wandb

wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: data-bigwig. Use `wandb login --relogin` to force relogin


True

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import json

import torch
from sklearn.metrics import r2_score
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
from torchsummary import summary

import os
import sys

## Utils and models

In [5]:
#get utils

sys.path.append(os.getcwd()+'/scripts')
# from Wandb_Utils import *
import Wandb_Utils as wu

In [6]:
###Networks
#single step
class RNN_Model(nn.Module):
    def __init__(self, config):
        super(RNN_Model, self).__init__()
        self.input_size = config.input_size
        self.hidden_size = config.hidden_size
        self.output_size = config.output_size
        self.num_layers = config.num_layers
        #default dropout = 0.1
        self.rnn = nn.RNN(self.input_size, self.hidden_size, self.num_layers, dropout = config.dropout, batch_first = True)
        self.linear = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.linear(out)
        return out
    
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.kaiming_uniform_(m.weight, nonlinearity= 'relu')
        m.bias.data.fill_(0.0)

###multiple time steps
#one to many RNN architechture
class Base_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout = 0.1):
        super(Base_RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(self.input_size, self.hidden_size, self.num_layers, dropout = dropout, batch_first = True)
        self.linear = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, in_hidden = None):
        if in_hidden == None:
            out, out_hidden = self.rnn(x)
        else:
            out, out_hidden = self.rnn(x, in_hidden)
        out = self.linear(out)
        return out, out_hidden

class RNN_Model_Multiple(nn.Module):
    def __init__(self, config):
        super(RNN_Model_Multiple, self).__init__()
        self.input_size = config.input_size
        self.hidden_size = config.hidden_size
        self.output_size = config.output_size
        self.num_layers = config.num_layers
        self.ntimesteps = config.ntimesteps
        self.base_rnn = Base_RNN(self.input_size, self.hidden_size, self.num_layers, config.dropout)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(self.input_size, self.output_size)
        
    def forward(self, x):
        out = x
        hidden = None
        
        #to store outputs
        outputs = []
        
        for t in range(self.ntimesteps):                
            #pass to rnn
            out, hidden = self.base_rnn(out, hidden)
            #pass to output layer
            outputs.append(self.linear(self.relu(out)))
            #skip connection
            out += x

        return torch.concat(outputs, 1)

#one to many RNN architechture
class Base_RNN_v2(nn.Module):
    def __init__(self, output_size, hidden_size, num_layers, dropout = 0.1):
        super(Base_RNN_v2, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(self.output_size, self.hidden_size, self.num_layers, dropout = dropout, batch_first = True)
        self.linear = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x, in_hidden = None):
        if in_hidden == None:
            out, out_hidden = self.rnn(x)
        else:
            out, out_hidden = self.rnn(x, in_hidden)
        out = self.linear(out)
        return out, out_hidden

class RNN_Model_Multiple_v2(nn.Module):
    def __init__(self, config):
        super(RNN_Model_Multiple_v2, self).__init__()
        self.input_size = config.input_size
        self.hidden_size = config.hidden_size
        self.output_size = config.output_size
        self.num_layers = config.num_layers
        self.ntimesteps = config.ntimesteps
        self.base_rnn = Base_RNN_v2(self.output_size, self.hidden_size, self.num_layers, config.dropout)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(self.input_size, self.output_size)

    def forward(self, x):
        #pass to first layer and activation
        x = self.relu(self.linear(x))

        #to store outputs
        outputs = []

        #set inputs to first RNN layer
        out = x
        hidden = None

        #loop through rnn layers
        for t in range(self.ntimesteps):                
            #pass to rnn
            out, hidden = self.base_rnn(out, hidden)
            #pass to output layer
            outputs.append(out)
            #skip connection
            out += x

        return torch.concat(outputs, 1)

# Model Optimization

In [7]:
#Define the sweep
sweep_config = {
    'name': 'Accelerating Hydrogen Oxidation Calculations (Concentration inputs, transformed data, Exclude pressure, Exclude H2 and O2 from model target, 5 steps (1,5, 15...))',
    'description': 'Model Optimization for Adam Optimizer',
    'method': 'bayes', #how sweep controller select hyperparameter
    'metric': {'name': 'loss',
               'goal': 'minimize'}
                }
sweep_config

{'name': 'Accelerating Hydrogen Oxidation Calculations (Concentration inputs, transformed data, Exclude pressure, Exclude H2 and O2 from model target, 5 steps (1,5, 15...))',
 'description': 'Model Optimization for Adam Optimizer',
 'method': 'bayes',
 'metric': {'name': 'loss', 'goal': 'minimize'}}

In [8]:
param_dict = {
    'batch_size': {
        'values': [8, 16, 32, 64, 128]
        },
    'timesteps': {
        'value': [1, 5, 15, 100, 200]
        },
    'nsample': {
        'value': 1500
        },
    'hidden_size': {
        'values': [20, 40, 60, 80, 100]
        },
    'num_layers': {
        'values': [2, 3, 4]
        },
    'dropout': {
        'values': [0, 0.05, 0.1, 0.15, 0.2]
    },
    'ntimesteps': {
        'value': 5
        },
    'epochs': {
        'value': 2500
        },
    'optimizer': {
        'value': 'Adam'
        },
    'lr': {
      'values': [5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4]
        },
    'weight_decay': {
        'values' : [0.05, 0.01, 0]
    },
    'beta1': {
        'values': [0.85, 0.9, 0.95]
        },
    'beta2': {
        'values': [0.9, 0.95, 0.99]
        },
    'mb_coeff': {
        'value': 0
        },
    'scheduling': {
        'values': [True, False]
       },
    'model': {
        'values': ['v1', 'v2']
    },
    'split': {
        'value': [0.7, 0.2, 0.1]
    },
    'dict_path':{
        'value': None
    },
    'mean_std_path': {
        'value': "ConcInputs_NoH2O2(taraget_only)_NoPressure_mean_std_1515100200.json"
    },
    'input_size':{
        'value': 9
    },
    'output_size':{
        'value': 7
    },
    'in_conc': {
        'value': True
    },
    'inputs_path': {
        'value' : 'input_98660.npy'
    },
    'outputs_dir': {
        'value': './Out_files_npy'
    },
    'in_h2_o2': {
        'value': True
    },
    'out_h2_o2': {
        'value': False
    },
    'in_pres': {
        'value': False
    }
    }
sweep_config['parameters'] = param_dict

In [9]:
sweep_config

{'name': 'Accelerating Hydrogen Oxidation Calculations (Concentration inputs, transformed data, Exclude pressure, Exclude H2 and O2 from model target, 5 steps (1,5, 15...))',
 'description': 'Model Optimization for Adam Optimizer',
 'method': 'bayes',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'batch_size': {'values': [8, 16, 32, 64, 128]},
  'timesteps': {'value': [1, 5, 15, 100, 200]},
  'nsample': {'value': 1500},
  'hidden_size': {'values': [20, 40, 60, 80, 100]},
  'num_layers': {'values': [2, 3, 4]},
  'dropout': {'values': [0, 0.05, 0.1, 0.15, 0.2]},
  'ntimesteps': {'value': 5},
  'epochs': {'value': 2500},
  'optimizer': {'value': 'Adam'},
  'lr': {'values': [0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]},
  'weight_decay': {'values': [0.05, 0.01, 0]},
  'beta1': {'values': [0.85, 0.9, 0.95]},
  'beta2': {'values': [0.9, 0.95, 0.99]},
  'mb_coeff': {'value': 0},
  'scheduling': {'values': [True, False]},
  'model': {'values': ['v1', 'v2']},
  'split': {'value

In [10]:
# Initialize Sweep
sweep_id = wandb.sweep(sweep_config, project="Hydrogen-Oxidation-Models-Optimization")

Create sweep with ID: mnzb7jr5
Sweep URL: https://wandb.ai/data-bigwig/Hydrogen-Oxidation-Models-Optimization/sweeps/mnzb7jr5


In [11]:
# sweep_id = '9s68j7y9'

In [12]:
def train_model(config = None):
  # tell wandb to get started
  with wandb.init(config=config):
  # access all HPs through wandb.config, so logging matches execution!
    config = wandb.config

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #Get data
    train_loader, val_loader, test_loader = wu.make_dataloaders(config)
    #obtaining mean and std of training set
    train_x_mean, train_x_std, train_y_mean, train_y_std = wu.load_mean_std(config.mean_std_path, train_loader)
    #initialize model
    if config.model == 'v1':
      model = RNN_Model_Multiple(config)
    elif config.model == 'v2':
      model = RNN_Model_Multiple_v2(config)

    #training
    model = model.to(device)
    mean_std = [train_x_mean.to(device), train_x_std.to(device), train_y_mean.to(device), train_y_std.to(device)]
    criterion  = nn.MSELoss()
    epochs, val_loss = wu.training(model, train_loader, val_loader, config, criterion, mean_std, device)
    wandb.log({'loss': val_loss})
    wandb.log({'num_epochs': epochs})

    # #saving best model from sweep
    # try:
    #   sweep_min_loss = wandb.Api().sweep(f'data-bigwig/Hydrogen-Oxidation-Models-Optimization/sweeps/{sweep_id}').best_run().summary_metrics['loss']
    # except Exception as e:
    #   print(f'Ooops! Probably first run. Save model anyways')
    #   sweep_min_loss = np.inf
      
    # # Saving State Dict
    # if val_loss <= sweep_min_loss:
    #   if config.dict_path != None:
    #     torch.save(model.state_dict(), config.dict_path)
    #     print('Model saving...')
    


In [ ]:
wandb.agent(sweep_id, train_model, project="Hydrogen-Oxidation-Models-Optimization", count=20)

wandb: Agent Starting Run: tn3q59ag with config:
wandb: 	batch_size: 128
wandb: 	beta1: 0.85
wandb: 	beta2: 0.99
wandb: 	dict_path: None
wandb: 	dropout: 0.15
wandb: 	epochs: 2500
wandb: 	hidden_size: 40
wandb: 	in_conc: True
wandb: 	in_h2_o2: True
wandb: 	in_pres: False
wandb: 	input_size: 9
wandb: 	inputs_path: input_98660.npy
wandb: 	lr: 0.05
wandb: 	mb_coeff: 0
wandb: 	mean_std_path: ConcInputs_NoH2O2(taraget_only)_NoPressure_mean_std_1515100200.json
wandb: 	model: v1
wandb: 	nsample: 1500
wandb: 	ntimesteps: 5
wandb: 	num_layers: 4
wandb: 	optimizer: Adam
wandb: 	out_h2_o2: False
wandb: 	output_size: 7
wandb: 	outputs_dir: ./Out_files_npy
wandb: 	scheduling: False
wandb: 	split: [0.7, 0.2, 0.1]
wandb: 	timesteps: [1, 5, 15, 100, 200]
wandb: 	weight_decay: 0.01
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▁
num_epochs,▁
train_loss,▃▇▄▂▄▂▄▃▃▄▄▂▂▄▂▄▂█▄▄▁▃▄▄▄▃▃▅█▃▄▄▂▂▅▄▄▃▃▂
val_loss,▂▇▇▂▃▁▃▄▃▂▃▁▃▂▁▃▂▆▂▄▄▁▃▃▃▃▂▃▅▂▂▂▁▂▄▂▆█▂▁
learning_rate,0.05
loss,1.58499
num_epochs,320
train_loss,0.70555
val_loss,1.00322


wandb: Agent Starting Run: 75v6u96o with config:
wandb: 	batch_size: 32
wandb: 	beta1: 0.95
wandb: 	beta2: 0.95
wandb: 	dict_path: None
wandb: 	dropout: 0.1
wandb: 	epochs: 2500
wandb: 	hidden_size: 80
wandb: 	in_conc: True
wandb: 	in_h2_o2: True
wandb: 	in_pres: False
wandb: 	input_size: 9
wandb: 	inputs_path: input_98660.npy
wandb: 	lr: 0.001
wandb: 	mb_coeff: 0
wandb: 	mean_std_path: ConcInputs_NoH2O2(taraget_only)_NoPressure_mean_std_1515100200.json
wandb: 	model: v2
wandb: 	nsample: 1500
wandb: 	ntimesteps: 5
wandb: 	num_layers: 2
wandb: 	optimizer: Adam
wandb: 	out_h2_o2: False
wandb: 	output_size: 7
wandb: 	outputs_dir: ./Out_files_npy
wandb: 	scheduling: True
wandb: 	split: [0.7, 0.2, 0.1]
wandb: 	timesteps: [1, 5, 15, 100, 200]
wandb: 	weight_decay: 0.05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
